In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

## 데이터 전처리 과정

In [3]:
# user_list = pd.read_csv('data/user_list_GH2.csv', index_col='Unnamed: 0')
path = 'data/'
train = pd.read_csv('{}train.csv'.format(path))
test = pd.read_csv('{}test.csv'.format(path))

In [4]:
#쓸모없는 칼럼 삭제
for df in (train,test):
    df.drop(['FLAG_MOBIL'], axis=1, inplace=True)   
    df.drop(columns=['index'], inplace=True)

### 1. ID컬럼 생성

In [5]:
# 개인정보를 나열한 칼럼 ID 추가 (만약 여기서 credit도 추가하게 되면 고유 ID가 12099명으로 증가함)
# 12099명으로 증가하는 경우는 유추하자면 credit 정보가 최신화되지 않았기 때문에 발생한다.
# 따라서 가장 최근의 credit을 user_list로 반영하기 위한 작업을 실시한다.
# 과거와 최근을 구별하는 방법은 bigin_month(카드 생성 후 기간)을 기준으로 시행한다.

def ID_col(df_1, df_2):
    """[데이터 셋 전처리 함수입니다.]

    Args:
        df_1 ([Dataset]): [데이터셋을 입력해주세요]
        df_2 ([Dataset]): [데이터셋을 입력해주세요]
    """
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']
    
    
    for name, dataset, in zip(names, df_list):
        dataset['ID'] = dataset['gender'].astype(str) + ' & ' + dataset['car'].astype(str) + ' & ' + dataset['reality'].astype(str) + ' & ' + \
                        dataset['child_num'].astype(str) + ' & ' + dataset['income_total'].astype(str) + ' & ' + dataset['income_type'].astype(str) + ' & ' + \
                        dataset['edu_type'].astype(str) + ' & ' + dataset['family_type'].astype(str) + ' & ' + dataset['house_type'].astype(str) + ' & ' + \
                        dataset['DAYS_BIRTH'].astype(str) + ' & ' + dataset['DAYS_EMPLOYED'].astype(str) + ' & ' + dataset['work_phone'].astype(str) + ' & ' + \
                        dataset['phone'].astype(str) + ' & ' + dataset['email'].astype(str)  + ' & ' + dataset['family_size'].astype(str)
        # 최신 credit반영을 위한 정렬
        dataset = dataset.sort_values(['begin_month', 'ID'], ascending=[False, True])
        
        # ID 중복되는 경우 True, 중복안된 고유값은 False로 설정하여 그 값의 개수를 확인한다
        # ID_TF 칼럼을 형성하여 True, False를 반영해준다.
        dataset['ID_TF'] = dataset.duplicated(['ID'])
        dataset = dataset.reset_index(inplace=False)
        print('해당 Data 내의 고유 ID 수는 {} 입니다.'.format(dataset['ID_TF'].value_counts()[0]))
        
        # card_num이라는 카드 개수를 추가하였음 (ID가 같은데 카드 개수가 여러 개인 사람을 대상으로 카드 개수를 구해서 데이터에 반영한다)
        print('Column(card_num)을 생성합니다.')
        for i in tqdm(range(len(dataset))):
            a = dataset.iloc[i]['ID']
            b = len(dataset[dataset['ID']==a])
            dataset.loc[(dataset['ID']==a), 'card_num'] =  b

        # 추출한 데이터프레임 (begin_month를 기준으로 최근의 credit을 반영)
        print('최근의 credit을 반영합니다')
        subset = dataset.query('ID_TF == False')
        subset = subset.reset_index(inplace=False)
        subset.drop(columns=['ID_TF'], inplace=True)
        
        # ID가 같은 사람을 뽑고 카드 보유기간의 평균을 뽑는다
        print('dataset에서 평균값을 추출합니다')
        for j in tqdm(range(len(dataset))):
            c = dataset.iloc[j]['ID']
            d = abs(int(dataset[dataset['ID']==c]['begin_month'].mean()))
            subset.loc[(subset['ID']==c), 'begin_month_mean'] =  d
        
        
        print('신규로 카드를 개설한 사람의 수 : {}'.format(len(dataset[dataset['begin_month']==0.0]['ID'].unique())))
        print('카드를 신규 개설하고 그 카드가 첫번째인 사람의 수 : {}'.format(len(subset[subset['begin_month_mean']==0.0])))
        print('dataset(user_list_{})을 저장합니다. \n'.format(name))    
        subset.to_csv('data/user_list_{}.csv'.format(name))
        
    del df_list, a, b, c, d, subset, names

In [6]:
ID_col(train,test)

해당 Data 내의 고유 ID 수는 8759 입니다.
Column(card_num)을 생성합니다.


100%|██████████| 26457/26457 [03:41<00:00, 119.31it/s]


최근의 credit을 반영합니다
dataset에서 평균값을 추출합니다


100%|██████████| 26457/26457 [03:07<00:00, 141.09it/s]


신규로 카드를 개설한 사람의 수 : 214
카드를 신규 개설하고 그 카드가 첫번째인 사람의 수 : 29
dataset(user_list_train)을 저장합니다.
해당 Data 내의 고유 ID 수는 5585 입니다.
Column(card_num)을 생성합니다.


100%|██████████| 10000/10000 [00:43<00:00, 231.76it/s]


최근의 credit을 반영합니다
dataset에서 평균값을 추출합니다


100%|██████████| 10000/10000 [00:39<00:00, 254.06it/s]


신규로 카드를 개설한 사람의 수 : 82
카드를 신규 개설하고 그 카드가 첫번째인 사람의 수 : 30
dataset(user_list_test)을 저장합니다.


In [34]:
path = 'data/'

user_list_train = pd.read_csv('{}user_list_train.csv'.format(path))
user_list_test = pd.read_csv('{}user_list_test.csv'.format(path))

### faimly_size 조정

In [35]:
def replace_value_family(df_1, df_2, num_of_outlier):
    """[Family_size의 outlier를 대치해주는 작업을 시행합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        num_of_outlier ([outlier_standard]): [family_size의 outlier 기준을 입력하세요]
    """
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    for dataset,name in zip(df_list,names):
        print('family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.')
        dataset.drop(columns=['child_num'], inplace=True)
        print('family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                        len(dataset.loc[dataset['family_size'] >= num_of_outlier])))
        sub = dataset.loc[dataset['family_size']>= num_of_outlier]
        for i in range(len(sub)):
            a = int(dataset[dataset['family_type'] == sub['family_type'].values[i]].mean()['family_size'])
            dataset.loc[(dataset['family_size']>= num_of_outlier), 'family_size'] = a
        
        print('family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                        len(dataset.loc[dataset['family_size'] >= num_of_outlier])))        # # family_size가 6을 초과하는 사람을 다른 값으로 대치하는 작업

        # family_size를 조정하고 난 후에 1인당 소득으로 total_income을 scale_down 해줌.
        print('1인당 소득으로 소득 수준을 조정합니다.')
        dataset['income_mean'] = dataset['income_total'] / dataset['family_size']
        print('{}데이터 셋 처리 완료.\n'.format(name))
    
    del sub,df_list, names
        
replace_value_family(user_list_train, user_list_test, 7)

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 5
family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
train데이터 셋 처리 완료.

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 4
family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
test데이터 셋 처리 완료.



### card_num 조정

In [36]:
# sns.boxenplot('card_num', data = user_list)
# plt.show()
# 카드 개수가 20개 이상인 사람을 대상으로 전처리 필요 판단

# col_corr_list(user_list, 'card_num')
# 해당 함수를 시행한 결과, card_num과 begin_month, credit의 상관성이 존재함을 확인

def replace_value_add(df_1,df_2,feature_name, standard_feature_1,  num_of_outlier):
    """[outlier를 대치하는 함수입니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        feature_name ([column name]): [outlier처리가 필요한 컬럼 이름을 입력해주세요]
        standard_feature_1 ([type]): [처리가 필요한 컬럼과 상관성이 높은 컬럼을 입력해주세요]
        num_of_outlier ([type]): [outlier 기준을 입력해주세요]
    """
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    for dataset,name in zip(df_list,names):
        print('{}이 {} 값 이상인 사람의 수 : {}'.format(feature_name ,num_of_outlier, 
                                                   len(dataset.loc[dataset[feature_name] >= num_of_outlier])))
        sub = dataset.loc[dataset[feature_name]>= num_of_outlier]
        for i in range(len(sub)):
            a = int(dataset[dataset[standard_feature_1] == sub[standard_feature_1].values[i]].mean()[feature_name])
            dataset.loc[(dataset[feature_name]>=num_of_outlier), feature_name] = a
        
        print('{}이 {} 값 이상인 사람의 수 : {}'.format(feature_name ,num_of_outlier, 
                                                   len(dataset.loc[dataset[feature_name] >= num_of_outlier])))   
        print('{}데이터 셋 처리 완료.\n'.format(name))

    del df_list, sub, a, names
    
replace_value_add(user_list_train, user_list_test, 'card_num', 'begin_month', 20)

card_num이 20 값 이상인 사람의 수 : 9
card_num이 20 값 이상인 사람의 수 : 0
train데이터 셋 처리 완료.

card_num이 20 값 이상인 사람의 수 : 0
card_num이 20 값 이상인 사람의 수 : 0
test데이터 셋 처리 완료.



### 나이, 고용연수 파생변수 생성

In [37]:
def year_month_total(df_1, df_2, column, column_name):
    """[나이, 고용연수 파생변수를 생성하기 위한 함수입니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        column ([생성할 컬럼의 기준 컬럼 명]): [생성할 컬럼의 기준 컬럼 명을 입력해주세요]
        column_name ([생성할 컬럼의 이름]): [연단위, 연+월단위로 표시된 파생변수 이름입니다.]
    """
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    if (column == 'DAYS_BIRTH') == True:    
        for dataset,name in zip(df_list,names):
            dataset['{}_year'.format(column_name)] = dataset[column].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset[column].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
        
            print('{}set에 {}_year, {}_total 컬럼을 생성하였습니다. \n'.format(name,column_name,column_name))
    
    else:
        for dataset,name in zip(df_list,names):
            print('{}set에 근무일자가 잘못기입된 {} cases에 대한 처리를 시작합니다.'.format(name, len(dataset[dataset['DAYS_EMPLOYED']>=0])))
            dataset['DAYS_EMPLOYED'].sort_values().value_counts()
            dataset['DAYS_EMPLOYED'] = dataset['DAYS_EMPLOYED'].replace(365243, 0)
            dataset.query('DAYS_EMPLOYED < 0')['DAYS_EMPLOYED'].sort_values()

            dataset['{}_year'.format(column_name)] = dataset[column].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset[column].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
        
            print('{}set에 {}_year, {}_total 컬럼을 생성하였습니다. \n'.format(name, column_name,column_name))
        
    del df_list,names
    

In [38]:
year_month_total(user_list_train, user_list_test,'DAYS_BIRTH','age')

trainset에 age_year, age_total 컬럼을 생성하였습니다. 

testset에 age_year, age_total 컬럼을 생성하였습니다. 



In [39]:
year_month_total(user_list_train, user_list_test,'DAYS_EMPLOYED','work')

trainset에 근무일자가 잘못기입된 1508 cases에 대한 처리를 시작합니다.
trainset에 work_year, work_total 컬럼을 생성하였습니다. 

testset에 근무일자가 잘못기입된 973 cases에 대한 처리를 시작합니다.
testset에 work_year, work_total 컬럼을 생성하였습니다. 



##### 근무일자는 근무연수로 표시한 경우 & 근무연수,개월,통합으로 정확한 값으로 표시한 경우 두가지로 진행해볼 것임

In [40]:
# # 근무일자를 제대로 표시하기 위한 작업
# print('근무일자가 잘못기입된 숫자 : {}'.format(len(user_list[user_list['DAYS_EMPLOYED']>=0])))

# #워킹일자 0보다 큰 값은 무효값으로 양수 전부 0으로 처리 후 work_year 칼럼 생성 (여기서는 work_year를 반올림한 값으로 처리)
# user_list['DAYS_EMPLOYED'].sort_values().value_counts()
# user_list['DAYS_EMPLOYED'] = user_list['DAYS_EMPLOYED'].replace(365243, 0)
# user_list.query('DAYS_EMPLOYED < 0')['DAYS_EMPLOYED'].sort_values()
# user_list['work_year'] = user_list['DAYS_EMPLOYED'].abs()/365
# user_list['work_year'] = user_list['work_year'].round()
# # user_list['work_year'].value_counts().sort_index()

### occpy_type Nan 값 처리

In [41]:
def occyp_type_nan(df_1, df_2):
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    for dataset,name in zip(df_list,names):
        dataset['occyp_type'] = dataset['occyp_type'].fillna('Nan')
        print('직업 컬럼의 Nan value 중 {} 명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.'.format(len(dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0)])))
        dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'jobless'
        print('나머지 값은 no_data로 처리합니다.')
        dataset.loc[dataset['occyp_type'] == 'Nan', 'occyp_type'] = 'no data'
        print('{}set의 직업 Nan value를 처리하였습니다.\n'.format(name))
        
    del df_list, names

occyp_type_nan(user_list_train, user_list_test)

직업 컬럼의 Nan value 중 1508 명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.
나머지 값은 no_data로 처리합니다.
trainset의 직업 Nan value를 처리하였습니다.

직업 컬럼의 Nan value 중 973 명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.
나머지 값은 no_data로 처리합니다.
testset의 직업 Nan value를 처리하였습니다.



In [42]:
names = ['train', 'test']

for name,dataset in zip(names, [user_list_train, user_list_test]):
    dataset.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED','level_0','index'], inplace=True)
    dataset.to_csv('data/final_set_{}.csv'.format(name))